In [83]:
import sys, os, time
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data, get_options

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
root = 'saved_models/'
names = '191119'
chpt = None
for i, name in enumerate(names):
    print(i,name)

0 1
1 9
2 1
3 1
4 1
5 9


In [158]:
#name = 'dspr-wae-adamp5p9_191118-222106'
name = 'dspr-wpp1e-2-adamp5p9-disclr3x_191119-110815'
#name = 'dspr-wgan-adamp5p9-disclr3x_191119-110855'
chpt = None
# chpt = 40
name

'dspr-wpp1e-2-adamp5p9-disclr3x_191119-110815'

In [159]:
args, (datasets, model) = train.load(os.path.join(root, name), load_data=get_data, load_model=get_model,
                                    saveroot='saved_models/',
                                    dataroot='../data/',
                                    load_last=True)
args.device = 'cpu'

Found 1 checkpoints. Using checkpoint_40.pth.tar
Loaded saved_models/dspr-wpp1e-2-adamp5p9-disclr3x_191119-110815/checkpoint_40.pth.tar
BCELoss()
Using gan-wt: 0.01
Using gen_types: {'rec'}
Loaded model_state from checkpoint


In [127]:
print(model)

Wasserstein_PP(
  (enc): Conv_Encoder(
    (conv): Sequential(
      (0): ConvLayer(
        (conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=True)
        (nonlin): PReLU(num_parameters=1)
      )
      (1): ConvLayer(
        (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=True)
        (nonlin): PReLU(num_parameters=1)
      )
      (2): ConvLayer(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (pool): MaxPool2d(

In [160]:
def _representation_function(x):
    x = np.transpose(x, (0,3,2,1))
    """Computes representation vector for input images."""
    output = model.encode(torch.Tensor(x))
    output = output.narrow(-1, 0, 16)
    return output.detach().numpy()


In [161]:
from metrics import mig
from metrics import metric_factor_vae
from metrics import metric_beta_vae
from ground_truth import dsprites
dataset = dsprites.DSprites()

In [162]:
metric_beta_vae.compute_beta_vae_sklearn(dataset, _representation_function, np.random, 10, 100, 100)

/Users/rookiejackal/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rookiejackal/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


{'train_accuracy': 0.52, 'eval_accuracy': 0.33}

In [163]:
metric_factor_vae.compute_factor_vae(dataset, _representation_function, np.random, 16, 60, 60, 60)

{'train_accuracy': 0.45,
 'eval_accuracy': 0.21666666666666667,
 'num_active_dims': 16}

In [164]:
mig.compute_mig(dataset, _representation_function, np.random, 60)

{'discrete_mig': 0.031784915664931856}

In [165]:
from metrics import dci
dci.compute_dci(dataset, _representation_function, np.random, 100, 100)

{'informativeness_train': 1.0,
 'informativeness_test': 0.192,
 'disentanglement': 0.04413885490516671,
 'completeness': 0.034079757368802296}

In [166]:
from metrics import irs
np.random.seed(10)
irs.compute_irs(dataset, _representation_function, np.random)

{'IRS': 0.752912036879518, 'num_active_dims': 15.372334}

In [167]:
from metrics import sap
np.random.seed(10)
sap.compute_sap(dataset, _representation_function, np.random, 100, 100, True)

{'SAP_score': 0.02185649869121685}

In [168]:
from metrics import modularity_explicitness
np.random.seed(10)
modularity_explicitness.compute_modularity_explicitness(dataset, _representation_function, np.random, 100, 100)

(5, 100)


/Users/rookiejackal/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rookiejackal/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/rookiejackal/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rookiejackal/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/rookiejackal/miniconda3/lib/python3

IndexError: index 37 is out of bounds for size 37

In [149]:
from metrics import unsupervised_metrics
unsupervised_metrics.unsupervised_metrics(dataset, _representation_function, np.random, 100)

{'gaussian_total_correlation': 0.9373908293496492,
 'gaussian_wasserstein_correlation': 0.9773025975951697,
 'gaussian_wasserstein_correlation_norm': 0.05868561022677615,
 'mutual_info_score': 1.000965264923025}